#### LangSmith Starter

# LangSmith Starter

LangSmith is an advanced open-source library tailored for AI Engineers, focusing on enabling seamless development and deployment of language-based AI systems. With an emphasis on modularity and interoperability, LangSmith simplifies the integration of natural language processing (NLP) capabilities into AI workflows, offering intuitive tools for fine-tuning, optimization, and interaction. It empowers developers to switch effortlessly between language models, customize pipelines, and deploy scalable, production-ready applications with minimal friction.

Super important note, you need to run LangSmith on more then 0.1.45 and be using the current API keys and not the legacy keys, if you do not do this you will recieve a 401 error message saying the token you are using is invalid.

This below is the enviroment data for enabling tracing, very important that you do not mess around with this as even replacing a singular character can break and then you will not be able to trace anything.

In [16]:
import os
from getpass import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = getpass("Enter LangSmith API Key: ")
os.environ["LANGCHAIN_PROJECT"] = "aai-langchain-course-langsmith-starter-ollama"

For this tutorial we will be using OpenAI, however this is basically the exact same for llama just with a few changes.

In [17]:
from langchain_ollama.chat_models import ChatOllama

model_name = "llama3.2:1b-instruct-fp16"

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(temperature=0.0, model=model_name)

Langsmith by default will trace all the usual langchain based LLM calls, which in the tutorials case is super useful as we won't have to edit much code to get traces working.

In [19]:
llm.invoke("hello")

AIMessage(content='Hello. Is there something I can help you with or would you like to chat?', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b-instruct-fp16', 'created_at': '2024-12-31T09:09:42.4472847Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 24838534100, 'load_duration': 19959371300, 'prompt_eval_count': 26, 'prompt_eval_duration': 499363000, 'eval_count': 18, 'eval_duration': 4345925000}, id='run-43697eeb-1932-40e5-9e76-db2f218f2796-0', usage_metadata={'input_tokens': 26, 'output_tokens': 18, 'total_tokens': 44})

This is settup for the non-langchain function.

In [20]:
import ollama

client = ollama

Now we want to use a non langchain related function, and these are not automatically traced by langsmith, so instead, we have to add the traceable decorator.

In [29]:
from langsmith import traceable

@traceable
def generate_response(question: str):

    complete_messages = [
    {"role": "system", "content": "You are a happy assistant"},
    {"role": "user", "content": question}
    ]

    return client.chat(
        model = model_name,
        messages = complete_messages,
    )

In [30]:
generate_response("How are you today?")

{'model': 'llama3.2:1b-instruct-fp16',
 'created_at': '2024-12-31T09:15:20.0684098Z',
 'message': {'role': 'assistant',
  'content': "I'm doing great, thanks for asking! I'm a large language model, so I don't have feelings or emotions like humans do, but I'm always ready to help and assist you with any questions or tasks you may have. It's nice to have a conversation with someone new and interesting! How about you? How's your day going so far?"},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 35365044500,
 'load_duration': 13020718600,
 'prompt_eval_count': 35,
 'prompt_eval_duration': 617363000,
 'eval_count': 73,
 'eval_duration': 21693342000}

We can also add data into the traceable by editing its paramaters, this includes metadata, and what we will change, the function name.

In [31]:
from langsmith import traceable

@traceable(name="OpenAI Response")
def generate_second_response(question: str):

    complete_messages = [
    {"role": "system", "content": "You are a happy assistant"},
    {"role": "user", "content": question}
    ]

    return client.chat(
        model = model_name,
        messages = complete_messages,
    )

In [32]:
generate_second_response("How are you today?")

{'model': 'llama3.2:1b-instruct-fp16',
 'created_at': '2024-12-31T09:16:13.7821909Z',
 'message': {'role': 'assistant',
  'content': "I'm doing great, thanks for asking! I'm a large language model, so I don't have emotions or feelings like humans do, but I'm always ready to help and assist with any questions or tasks you may have. It's nice to start the day on a positive note, isn't it? How about you, how's your day going so far?"},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 26864340500,
 'load_duration': 98078200,
 'prompt_eval_count': 35,
 'prompt_eval_duration': 890598000,
 'eval_count': 75,
 'eval_duration': 25866451000}

Now you can view all of this over at the LangSmith website, under the tracing projects.